> **Exercise 1: Mixing Patterns and Assortativity.**
>
> * For each node, compute the fraction of edges that connect to a node that works in the same top field. Find the average value across all nodes.
> * Create a new graph, with the same nodes and edges, but where the association between nodes and field is shuffled. Compute the measure above for this randomized graph.
> * Repeat the point above 100 times (at least). Plot the distribution of the values obtained and compare it with the value you have found for the real graph. Is the chance to connect to a member of the same field significantly higher than it would be by chance?
> * Compute the assortativity coefficient with respect to author's field. How do you interpret the value you obtain? (__Hint__: See [this paper](https://nbviewer.org/github/suneman/socialgraphs2019/blob/master/lectures/Week5.ipynb), eq (2)). **Important**: here I do not want you to use the NetworkX implementation, but rather to implement your own version of the measure.
> * Is the graph assortative with respect to the degree? (e.g. do high-degree scientists tend to link to other high-degree scientists, and low-degree scientists to other low-degree scientists?). Provide an interpretation of your answer.
> * _Optional:_ Estimate the gender of each author from their name, using the [World Gender Name Dictionary](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/MSEGSJ). Repeat the analysis above to study the assortativity of the network by gender rather than by field. What do you observe?


In [69]:
import networkx as nx
import pandas as pd
import pickle as pkl
import requests as rq
from semanticscholar import SemanticScholar
import re

In [80]:
authors = pd.read_csv('all_authors.csv', index_col=0)
papers = pd.read_csv('all_papers.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [81]:

# import author data
# We have a mistake in the data, so each author only has their top field listed
# {"id1" = 123123123,"id2" = 123123123, "weight" = 3"}

pattern = '[a-zA-Z]+' # regex pattern to match only letters

# list of dicts with authorid 1 and authorid 2 and how many times they coauthored each other
edgelist = []
for id in authors["id"]:
    for paperindex in range(len(papers)):
        paper = papers.iloc[paperindex]['authors']
        authorlist = re.sub(pattern,"",paper[1:-1].replace("'","")).split(",")
        paper_authors = [int(x) for x in authorlist if x not in ["", " ", "  "]]
        if id in paper_authors:
            # drop paper from dataframe so we don't recount it
            papers.drop(paperindex, inplace=True)
            for author in paper_authors:
                if author != id:
                    for dic in edgelist:
                        if dic["id1"] == id and dic["id2"] == author:
                            dic["weight"] += 1
                        elif dic["id1"] == author and dic["id2"] == id:
                            dic["weight"] += 1
                        else:
                            edgelist.append({"id1": id, "id2": author, "weight": 1})

edgelist


KeyError: '[1351208] not found in axis'